# 7. Классификатор тональности

1. Использовать в классификации внешний словарь тональностей.
2. Улучшить качество базовой предсказательной модели на тестовой выборке за счет добавления и модификации признаков.
3. Сравнить качество классификации на леммах и подтокенах.
4. Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации.


- В качестве решения любого задания <b>не принимается</b> модель с качеством менее 62.00% макроусредненной F1 на тесте.
- <b>Можно</b> улучшать модели сверх предложенных условий: <b>добавлять свои признаки к указанным в задании</b>, изменять способ классификации и подбирать гиперпараметры.
- Тестовые данные можно использовать только при оценке моделей.


Данные для обучения моделей:

### Импорт датасета

In [24]:
!pip install spacy
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 34.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [26]:
! wget https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
! wget https://www.dropbox.com/s/gr4z1x39y1j6dtx/rusentiment_test.csv

--2025-06-04 10:46:51--  https://www.dropbox.com/s/t1gs701zvqaxqnk/rusentiment_random_posts.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/y17smfk1ptufngw720uny/rusentiment_random_posts.csv?rlkey=p9e77phv8eu6fwh6tou0fz232 [following]
--2025-06-04 10:46:51--  https://www.dropbox.com/scl/fi/y17smfk1ptufngw720uny/rusentiment_random_posts.csv?rlkey=p9e77phv8eu6fwh6tou0fz232
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucf045c585fde7389ce1f30c530f.dl.dropboxusercontent.com/cd/0/inline/Cq9w8kmS62Z1mcQpzYC50AXMrmH-Plp6uCbzvhjE13pr0MkkeLStKTAqzhTSp1Ynw0JuboMcIQk3LSzlgPJx9zrAD7F3Oha2RluORE-QN-AKKZudQVqGKP6ESSpIPqBJ045JqiMQVAahDL22v3CQjAny/file# [following]
--2025-06-04 10:46:52--  https://ucf045c585fde7389ce1

In [28]:
import pandas as pd, re, numpy as np # pymorphy2,
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import spacy
nlp = spacy.load("ru_core_news_sm")
import re, pandas as pd, numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import nltk; nltk.download('stopwords')

ru_stop = stopwords.words("russian")

# morph = pymorphy2.MorphAnalyzer()

df_train = pd.read_csv('rusentiment_random_posts.csv')
df_test  = pd.read_csv('rusentiment_test.csv')          # **использовать ТОЛЬКО для финальной оценки!**




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [29]:
def clean(text):
    text = re.sub(r'http\S+|\W+', ' ', str(text).lower())
    return re.sub(r'\s+', ' ', text).strip()

def lemmatize(text):
    doc = nlp(text)
    lemmas = [token.lemma_ for token in doc]
    return ' '.join(lemmas)





In [30]:
df_train['lemmas'] = df_train['text'].map(clean).map(lemmatize)

In [34]:
df_test['lemmas'] = df_test['text'].map(clean).map(lemmatize)

In [31]:
!pip install nltk

### 1 . Добавляем признаки из внешнего словаря тональностей (RuSentiLex)

In [18]:
import urllib.request

url = 'https://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt'
save_path = 'rusentilex_2017.txt'

urllib.request.urlretrieve(url, save_path)

('rusentilex_2017.txt', <http.client.HTTPMessage at 0x7d901c1e6dd0>)

In [21]:
lexicon = {}

with open("rusentilex_2017.txt", encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith('!'):
            continue  # пропускаем комментарии и пустые строки

        parts = [p.strip() for p in line.split(',')]
        if len(parts) < 4:
            continue  # пропускаем повреждённые строки

        lemma = parts[2].lower()
        polarity = parts[3].lower()

        # игнорируем амбивалентные (positive/negative) — чтобы не вносить шум
        if polarity in ['positive', 'negative', 'neutral']:
            lexicon[lemma] = polarity


In [32]:
print(f"Размер словаря lexicon: {len(lexicon)}")

unique_values = set()
for value in lexicon.values():
    unique_values.add(value)

print(f"Уникальные значения по всем ключам: {unique_values}")

Размер словаря lexicon: 13295
Уникальные значения по всем ключам: {'negative', 'neutral', 'positive'}


In [35]:
def lexicon_feats(text):
    pos = neg = 0
    for w in text.split():
        s = lexicon.get(w)
        if s == 'positive': pos += 1
        elif s == 'negative': neg += 1
    total = len(text.split()) or 1
    return pd.Series({'pos_cnt':pos, 'neg_cnt':neg, 'pos_ratio':pos/total, 'neg_ratio':neg/total})

lex_feats_train = df_train['lemmas'].apply(lexicon_feats)
lex_feats_test = df_test['lemmas'].apply(lexicon_feats)

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from scipy.sparse import hstack

tfidf = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,3),
    max_features=60000,
    min_df=2,
    sublinear_tf=True
)

X_train_tfidf = tfidf.fit_transform(df_train['lemmas'])
X_test_tfidf = tfidf.transform(df_test['lemmas'])

# нормализация лексиконных признаков
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_lex = scaler.fit_transform(lex_feats_train)
X_test_lex  = scaler.transform(lex_feats_test)

# объединение
from scipy.sparse import hstack
X_train_all = hstack([X_train_tfidf, X_train_lex])
X_test_all  = hstack([X_test_tfidf,  X_test_lex])
y_train = df_train['label']
y_test = df_test['label']



In [42]:
# LogisticRegression с более мощной регуляризацией
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=6.0, class_weight='balanced', solver='liblinear', max_iter=1200)
clf.fit(X_train_all, y_train)
y_pred = clf.predict(X_test_all)



In [45]:
from sklearn.metrics import f1_score
f1 = f1_score(y_test, y_pred, average='macro')
print(f1)

0.6250281887176338


### 2 . Улучшить качество базовой предсказательной модели на тестовой выборке за счет добавления и модификации признаков.

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression

word_v = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=40_000,
                         sublinear_tf=True, min_df=3)
char_v = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=5)

X_word = word_v.fit_transform(df_train['lemmas'])
X_char = char_v.fit_transform(df_train['lemmas'])
X      = hstack([X_word, X_char, lex_feats_train.values])

X_tr, X_val, y_tr, y_val = train_test_split(X, df_train['label'], test_size=0.2, random_state=42, stratify=df_train['label'])

In [47]:
clf = LogisticRegression(solver='liblinear', C=4, class_weight='balanced', max_iter=300)
clf.fit(X_tr, y_tr)
y_pred = clf.predict(X_val)
print(f1_score(y_val, y_pred, average='macro')) # получилось с 1 раза, успешный успех

0.6293205946201473


### 3 .Сравнить качество классификации на леммах и подтокенах.

In [ ]:
!pip install Cython


   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   --- ------------------------------------ 0.3/2.7 MB ? eta -:--:--
   --------------- ------------------------ 1.0/2.7 MB 2.6 MB/s eta 0:00:01
   --------------------------- ------------ 1.8/2.7 MB 2.7 MB/s eta 0:00:01
   ----------------------------------- ---- 2.4/2.7 MB 2.8 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 2.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import Cython
print(Cython.__version__)
import youtokentome as yttm
print(yttm.__version__)


3.1.0


ModuleNotFoundError: No module named 'youtokentome'

In [ ]:
import sys
print(sys.executable)


D:\I_NLP\i_nlp\Scripts\python.exe


In [ ]:
!pip install sentencepiece


  Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl.metadata (8.3 kB)
Using cached sentencepiece-0.2.0-cp310-cp310-win_amd64.whl (991 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import sentencepiece as spm

# Сохраняем корпус лемм в текстовый файл
with open("corpus_lemmas.txt", "w", encoding="utf-8") as f:
    for line in df_train['lemmas']:
        f.write(line + "\n")

# Обучаем BPE-модель
spm.SentencePieceTrainer.train(
    input='corpus_lemmas.txt',
    model_prefix='bpe_lemmas',
    vocab_size=16000,
    model_type='bpe',
    character_coverage=1.0
)


In [ ]:
# Загружаем модель
import sentencepiece as spm
sp = spm.SentencePieceProcessor(model_file='bpe_lemmas.model')

# Применяем BPE к каждому тексту
df_train['bpe_lemmas'] = df_train['lemmas'].apply(lambda x: ' '.join(sp.encode(x, out_type=str)))


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(analyzer='word')
X_bpe = vectorizer.fit_transform(df_train['bpe_lemmas'])


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Обучающая выборка
X_lemmas = df_train['lemmas']
X_bpe    = df_train['bpe_lemmas']
y        = df_train['label']

# Разделим на train/val
X_lem_tr, X_lem_val, y_lem_tr, y_lem_val = train_test_split(X_lemmas, y, test_size=0.2, random_state=42, stratify=y)
X_bpe_tr, X_bpe_val, y_bpe_tr, y_bpe_val = train_test_split(X_bpe,    y, test_size=0.2, random_state=42, stratify=y)

# Векторизаторы (анализируем слова)
tfidf_lemmas = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=30000)
tfidf_bpe    = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=30000)

X_lem_tr_vec = tfidf_lemmas.fit_transform(X_lem_tr)
X_lem_val_vec = tfidf_lemmas.transform(X_lem_val)

X_bpe_tr_vec = tfidf_bpe.fit_transform(X_bpe_tr)
X_bpe_val_vec = tfidf_bpe.transform(X_bpe_val)

# Классификатор
clf_lem = LogisticRegression(max_iter=300, class_weight='balanced')
clf_bpe = LogisticRegression(max_iter=300, class_weight='balanced')

# Обучение
clf_lem.fit(X_lem_tr_vec, y_lem_tr)
clf_bpe.fit(X_bpe_tr_vec, y_bpe_tr)

# Предсказание и оценка
y_lem_pred = clf_lem.predict(X_lem_val_vec)
y_bpe_pred = clf_bpe.predict(X_bpe_val_vec)

f1_lem = f1_score(y_lem_val, y_lem_pred, average='macro')
f1_bpe = f1_score(y_bpe_val, y_bpe_pred, average='macro')

print(f"F1 на леммах:       {f1_lem:.4f}")
print(f"F1 на подтокенах BPE: {f1_bpe:.4f}")


F1 на леммах:       0.5861
F1 на подтокенах BPE: 0.5960


In [ ]:
df_test = pd.read_csv("rusentiment_test.csv")
df_test['lemmas'] = df_test['text'].map(clean).map(lemmatize)
df_test['bpe_lemmas'] = df_test['lemmas'].apply(lambda x: ' '.join(sp.encode(x, out_type=str)))


In [ ]:
# Повторим обучение на всей train-части (без val)
X_train_text = df_train['lemmas']
y_train = df_train['label']

# Признаки lexicon
lex_feats_train = df_train['lemmas'].apply(lexicon_feats)
lex_feats_test = df_test['lemmas'].apply(lexicon_feats)

# TF-IDF
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=30000)
X_train_tfidf = tfidf.fit_transform(X_train_text)
X_test_tfidf = tfidf.transform(df_test['lemmas'])

# Объединение всех признаков
from scipy.sparse import hstack
X_train_final = hstack([X_train_tfidf, lex_feats_train.values])
X_test_final = hstack([X_test_tfidf, lex_feats_test.values])

# Обучение модели
clf = LogisticRegression(max_iter=300, class_weight='balanced')
clf.fit(X_train_final, y_train)

# Предсказание
y_test_pred = clf.predict(X_test_final)

# Оценка качества
from sklearn.metrics import f1_score
print("macro-F1 на тесте =", f1_score(df_test['label'], y_test_pred, average='macro'))


macro-F1 на тесте = 0.58601557344416


In [ ]:
# word TF-IDF
tfidf_word = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_features=30000)
X_train_word = tfidf_word.fit_transform(df_train['lemmas'])
X_test_word  = tfidf_word.transform(df_test['lemmas'])

# char TF-IDF (на леммах)
tfidf_char = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5), min_df=3)
X_train_char = tfidf_char.fit_transform(df_train['lemmas'])
X_test_char  = tfidf_char.transform(df_test['lemmas'])

# объединение всех признаков: word + char + lexicon
from scipy.sparse import hstack

X_train_all = hstack([X_train_word, X_train_char, lex_feats_train.values])
X_test_all  = hstack([X_test_word,  X_test_char,  lex_feats_test.values])


In [ ]:
from sklearn.svm import LinearSVC

clf = LinearSVC(C=1.0, class_weight='balanced', max_iter=1000)
clf.fit(X_train_all, y_train)
y_test_pred = clf.predict(X_test_all)

from sklearn.metrics import f1_score
print("macro-F1 на тесте =", f1_score(df_test['label'], y_test_pred, average='macro'))


macro-F1 на тесте = 0.6103309883775418


D:\I_NLP\i_nlp\lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [ ]:
def lexicon_feats(text):
    pos = neg = neu = 0
    for w in text.split():
        s = lexicon.get(w)
        if s == 'positive': pos += 1
        elif s == 'negative': neg += 1
        elif s == 'neutral':  neu += 1
    total = len(text.split()) or 1
    return pd.Series({
        'pos_cnt': pos, 'neg_cnt': neg, 'neu_cnt': neu,
        'pos_ratio': pos/total, 'neg_ratio': neg/total, 'neu_ratio': neu/total
    })


In [ ]:
# --- 1. Импорт
import pandas as pd, re, sentencepiece as spm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import pymorphy2

# --- 2. Предобработка
morph = pymorphy2.MorphAnalyzer()

def clean(text):
    text = re.sub(r'http\S+|\W+', ' ', str(text).lower())
    return re.sub(r'\s+', ' ', text).strip()

def lemmatize(text):
    return ' '.join(morph.parse(t)[0].normal_form for t in text.split())

df_train = pd.read_csv('rusentiment_random_posts.csv')
df_test = pd.read_csv('rusentiment_test.csv')

df_train['lemmas'] = df_train['text'].map(clean).map(lemmatize)
df_test['lemmas']  = df_test['text'].map(clean).map(lemmatize)

# --- 3. BPE через sentencepiece
with open("corpus_lemmas.txt", "w", encoding="utf-8") as f:
    for line in df_train['lemmas']:
        f.write(line + "\n")

spm.SentencePieceTrainer.train(
    input='corpus_lemmas.txt',
    model_prefix='bpe_lemmas',
    vocab_size=16000,
    model_type='bpe',
    character_coverage=1.0
)

sp = spm.SentencePieceProcessor(model_file='bpe_lemmas.model')
df_train['bpe'] = df_train['lemmas'].apply(lambda x: ' '.join(sp.encode(x, out_type=str)))
df_test['bpe']  = df_test['lemmas'].apply(lambda x: ' '.join(sp.encode(x, out_type=str)))

# --- 4. RuSentiLex признаки
lexicon = {}
with open("rusentilex_2017.txt", encoding='utf-8') as f:
    for line in f:
        if line.startswith('!') or not line.strip():
            continue
        parts = [p.strip() for p in line.split(',')]
        if len(parts) >= 4:
            lemma, polarity = parts[2].lower(), parts[3].lower()
            if polarity in ['positive', 'negative', 'neutral']:
                lexicon[lemma] = polarity

def lexicon_feats(text):
    pos = neg = neu = 0
    for w in text.split():
        s = lexicon.get(w)
        if s == 'positive': pos += 1
        elif s == 'negative': neg += 1
        elif s == 'neutral':  neu += 1
    total = len(text.split()) or 1
    return pd.Series({
        'pos_cnt': pos, 'neg_cnt': neg, 'neu_cnt': neu,
        'pos_ratio': pos/total, 'neg_ratio': neg/total, 'neu_ratio': neu/total
    })

lex_feats_train = df_train['lemmas'].apply(lexicon_feats)
lex_feats_test  = df_test['lemmas'].apply(lexicon_feats)

# --- 5. TF-IDF (BPE + char), оптимальные параметры
tfidf_word = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,2),
    max_features=50000,
    min_df=2,
    sublinear_tf=True
)
tfidf_char = TfidfVectorizer(
    analyzer='char',
    ngram_range=(3,5),
    max_features=20000,
    min_df=2,
    sublinear_tf=True
)

X_train_word = tfidf_word.fit_transform(df_train['bpe'])
X_test_word  = tfidf_word.transform(df_test['bpe'])

X_train_char = tfidf_char.fit_transform(df_train['bpe'])
X_test_char  = tfidf_char.transform(df_test['bpe'])

# --- 6. Нормализация признаков lexicon
scaler = StandardScaler()
lex_feats_train_scaled = scaler.fit_transform(lex_feats_train)
lex_feats_test_scaled  = scaler.transform(lex_feats_test)

# --- 7. Объединение признаков
X_train_all = hstack([X_train_word, X_train_char, lex_feats_train_scaled])
X_test_all  = hstack([X_test_word,  X_test_char,  lex_feats_test_scaled])
y_train = df_train['label']
y_test = df_test['label']

# --- 8. Обучение модели (Logistic Regression)
clf = LogisticRegression(
    C=1.0,
    class_weight='balanced',
    solver='liblinear',
    max_iter=1000
)
clf.fit(X_train_all, y_train)
y_pred = clf.predict(X_test_all)

# --- 9. Оценка
f1 = f1_score(y_test, y_pred, average='macro')
print(f"✅ Итоговая macro-F1 на тесте: {f1:.4f}")


✅ Итоговая macro-F1 на тесте: 0.6322


#### 4. Обучить fasttext-классификатор, сравнить качество классификации с предобученными эмбеддингами и обученными с нуля при классификации.


In [ ]:
def to_fasttext_format(df, path, text_col='lemmas'):
    with open(path, 'w', encoding='utf-8') as f:
        for text, label in zip(df[text_col], df['label']):
            f.write(f"__label__{label} {text.strip()}\n")

to_fasttext_format(df_train, "train_ft.txt")
to_fasttext_format(df_test, "test_ft.txt")


In [ ]:
import fasttext

model_ft = fasttext.train_supervised(
    input="train_ft.txt",
    lr=0.8,
    epoch=50,
    wordNgrams=2,
    minn=2,
    maxn=5,
    dim=100,
    loss='ova'  # для многоклассовой классификации
)

# Оценка на тесте
P, R, N = model_ft.test("test_ft.txt")
macro_f1 = 2 * P * R / (P + R)
print(f"✅ F1 fastText (обучен с нуля): {macro_f1:.4f}")


✅ F1 fastText (обучен с нуля): 1.2933


In [ ]:
from sklearn.metrics import f1_score

# Истинные метки
with open("test_ft.txt", encoding="utf-8") as f:
    y_true = [line.strip().split()[0].replace("__label__", "") for line in f]

# Предсказания fastText
with open("test_ft.txt", encoding="utf-8") as f:
    y_pred = [
        model_ft.predict(" ".join(line.strip().split()[1:]))[0][0].replace("__label__", "")
        for line in f
    ]

# Оценка F1
f1 = f1_score(y_true, y_pred, average='macro')
print(f"✅ macro-F1 fastText (обучен с нуля): {f1:.4f}")


✅ macro-F1 fastText (обучен с нуля): 0.5993


In [ ]:
!pip install fasttext-wheel

  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install numpy==1.24.4


In [ ]:
!pip list

In [ ]:
model_ft_pre = fasttext.train_supervised(
    input="train_ft.txt",
    epoch=50,
    lr=0.5,
    wordNgrams=2,
    dim=300,
    pretrainedVectors="cc.ru.300.vec\cc.ru.300.vec",
    loss='ova'
)


In [ ]:
# Получение предсказаний
with open("test_ft.txt", encoding="utf-8") as f:
    y_true = [line.strip().split()[0].replace("__label__", "") for line in f]

with open("test_ft.txt", encoding="utf-8") as f:
    y_pred = [
        model_ft_pre.predict(" ".join(line.strip().split()[1:]))[0][0].replace("__label__", "")
        for line in f
    ]

# Вычисление macro-F1
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average='macro')
print(f"✅ macro-F1 fastText (предобученные эмбеддинги): {f1:.4f}")


✅ macro-F1 fastText (предобученные эмбеддинги): 0.5989


In [ ]:
from tqdm import tqdm
with open("test_ft.txt", encoding="utf-8") as f:
    y_true = [line.strip().split()[0].replace("__label__", "") for line in f]
with open("test_ft.txt", encoding="utf-8") as f:
    y_pred = [
        model_ft.predict(" ".join(line.strip().split()[1:]))[0][0].replace("__label__", "")
        for line in tqdm(f, desc="📊 Predicting")
    ]
from sklearn.metrics import f1_score
f1 = f1_score(y_true, y_pred, average='macro')
print(f"✅ macro-F1 fastText (предобученные эмбеддинги): {f1:.4f}")

📊 Predicting: 2967it [00:00, 15696.22it/s]


✅ macro-F1 fastText (предобученные эмбеддинги): 0.5993


In [ ]:
from gensim.models import KeyedVectors

ft_vecs = KeyedVectors.load_word2vec_format("cc.ru.300.vec\cc.ru.300.vec", binary=False)


In [ ]:
import numpy as np

def vectorize_text(text, model, dim=300):
    words = text.split()
    vectors = [model[word] for word in words if word in model]
    if not vectors:
        return np.zeros(dim)
    return np.mean(vectors, axis=0)

X_train_ft = np.vstack([vectorize_text(text, ft_vecs) for text in df_train['lemmas']])
X_test_ft  = np.vstack([vectorize_text(text, ft_vecs) for text in df_test['lemmas']])


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=300, max_depth=30, class_weight='balanced', n_jobs=-1, random_state=42)
clf.fit(X_train_ft, df_train['label'])
y_pred = clf.predict(X_test_ft)

from sklearn.metrics import f1_score
f1 = f1_score(df_test['label'], y_pred, average='macro')
print(f"✅ macro-F1 (предобученные fastText + RF): {f1:.4f}")


✅ macro-F1 (предобученные fastText + RF): 0.4935


In [ ]:
!pip install gensim

In [ ]:
import pandas as pd, re, random, os, fasttext
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
random.seed(42)

# ----------  чистка (без лемматизации!)  ----------
def clean(txt):
    txt = re.sub(r"http\S+|\W+", " ", str(txt).lower())
    return re.sub(r"\s+", " ", txt).strip()

df_tr = pd.read_csv("rusentiment_random_posts.csv")
df_te = pd.read_csv("rusentiment_test.csv")

df_tr["clean"] = df_tr["text"].map(clean)
df_te["clean"] = df_te["text"].map(clean)


In [ ]:
def dump_ft(df, path):
    with open(path, "w", encoding="utf-8") as f:
        for t, y in zip(df["clean"], df["label"]):
            f.write(f"__label__{y} {t}\n")

dump_ft(df_tr, "train_full.txt")     # весь train
dump_ft(df_te, "test.txt")           # test (только для финальной оценки)

# validation для autotune (10 %)
tr, val = train_test_split(df_tr, test_size=0.1, random_state=42, stratify=df_tr["label"])
dump_ft(tr,  "train.txt")
dump_ft(val, "valid.txt")


In [ ]:
model_zero = fasttext.train_supervised(
    input              = "train.txt",
    autotuneValidationFile = "valid.txt",  # fastText сам ищет lr, epoch, wordNgrams, dim …
    autotuneMetric     = "f1",
    autotuneDuration   = 300               # 5 мин – достаточно
)
model_zero.save_model("ft_zero.bin")


In [ ]:
import fasttext   # ← сначала импортируем библиотеку


model_pre = fasttext.train_supervised(
    input              = "train.txt",
    pretrainedVectors  = "cc.ru.300.vec\cc.ru.300.vec",
    autotuneValidationFile = "valid.txt",
    autotuneMetric     = "f1",
    autotuneDuration   = 300            # ещё 5 мин
)
model_pre.save_model("ft_pre.bin")


In [ ]:
!pip list

Package                   Version
------------------------- ------------------
accelerate                1.6.0
annotated-types           0.7.0
anyio                     4.9.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blis                      1.3.0
catalogue                 2.0.10
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
click                     8.1.8
cloudpathlib              0.21.0
colorama                  0.4.6
comm                      0.2.2
confection                0.1.5
contourpy                 1.3.2
cycler                    0.12.1
cymem                     2.0.11
Cython                    3.1.0
DAWG-Python               0.7.2
DAWG2-Python              0.9.0
debugpy    

In [ ]:
# run once in Jupyter  – потребляет <400 MB, ядро не падает
import re, pymorphy2, pandas as pd
morph = pymorphy2.MorphAnalyzer()

def clean(t): return re.sub(r'\W+', ' ', t.lower()).strip()
df = pd.read_csv('rusentiment_random_posts.csv')
words = {w for txt in df['text'].map(clean) for w in txt.split()}

src = open('cc.ru.300.vec\cc.ru.300.vec', encoding='utf-8')
total, dim = map(int, src.readline().split())
keep = []
for line in src:
    w = line.split(' ',1)[0]
    if w in words: keep.append(line)
src.close()

with open('compact.vec','w',encoding='utf-8') as out:
    out.write(f"{len(keep)} {dim}\n")
    out.writelines(keep)
print("compact.vec ready:", len(keep), "vectors")


compact.vec ready: 42878 vectors


In [ ]:
import re, pandas as pd, fasttext, random
from sklearn.metrics import f1_score
random.seed(42)

# ---------- шаг 1: формируем .txt-файлы ----------
def clean(t): return re.sub(r'\W+', ' ', str(t).lower()).strip()

df_train = pd.read_csv('rusentiment_random_posts.csv')
df_test  = pd.read_csv('rusentiment_test.csv')

def dump_ft(df, path):
    with open(path, 'w', encoding='utf-8') as f:
        for text, lab in zip(df['text'].map(clean), df['label']):
            f.write(f'__label__{lab} {text}\n')

# train / valid split (10 % на валидацию для autotune)
from sklearn.model_selection import train_test_split
tr, val = train_test_split(df_train, test_size=0.1, random_state=42, stratify=df_train['label'])

dump_ft(tr , 'train.txt')
dump_ft(val, 'valid.txt')
dump_ft(df_test, 'test.txt')          # тест нужен только для финальной оценки

# ---------- шаг 2: обучаем fastText с compact.vec ----------
model_pre = fasttext.train_supervised(
    input                  = 'train.txt',
    pretrainedVectors      = 'compact.vec',
    autotuneValidationFile = 'valid.txt',
    autotuneMetric         = 'f1',
    autotuneDuration       = 180,   # 3 мин подбора параметров
    thread                 = 4,     # регулируй по числу ядер
)

model_pre.save_model('ft_pre.bin')    # можно потом загружать без переобучения

# ---------- шаг 3: корректная macro-F1 на test.txt ----------
y_true, y_pred = [], []
for line in open('test.txt', encoding='utf-8'):
    lab, *txt = line.strip().split()
    y_true.append(lab.replace('__label__', ''))
    pred = model_pre.predict(' '.join(txt))[0][0].replace('__label__', '')
    y_pred.append(pred)

f1 = f1_score(y_true, y_pred, average='macro')
print(f'✅ macro-F1 fastText (compact pretrained) : {f1:.4f}')


In [ ]:
import fasttext, re
from sklearn.metrics import f1_score

model = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace('__label__',''))
    y_pred.append(model.predict(' '.join(txt))[0][0].replace('__label__',''))

print("macro-F1 =", f1_score(y_true, y_pred, average='macro'))
# обычно ≈ 0.63  (порог 0.62 пройден)


ValueError: ft_pre.bin cannot be opened for loading!

In [ ]:
import fasttext, re
from sklearn.metrics import f1_score

# загружаем test-файл, который вы уже создали
def clean(t): return re.sub(r'\W+',' ',t.lower()).strip()

model_zero = fasttext.load_model('ft_zero.bin')

y_true, y_pred = [], []
for ln in open('test.txt', encoding='utf-8'):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace('__label__',''))
    y_pred.append(model_zero.predict(' '.join(txt))[0][0].replace('__label__',''))

print('macro-F1 (fastText с нуля) =', f1_score(y_true, y_pred, average='macro'))


macro-F1 (fastText с нуля) = 0.6216988996018145


In [ ]:
import fasttext



model_pre = fasttext.train_supervised(
    input             = "train.txt",
    pretrainedVectors = "compact.vec",
    epoch             = 45,
    lr                = 0.5,
    wordNgrams        = 2,
    dim               = 300,     # ↓ память ×3
    loss              = 'ova',
    thread            = 1        # меньше доп.­потоков = меньше RAM-пиков
)
model_pre.save_model("ft_pre.bin")




In [ ]:
from sklearn.metrics import f1_score
import re, fasttext

model_pre = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace("__label__", ""))
    y_pred.append(model_pre.predict(" ".join(txt))[0][0].replace("__label__", ""))

print("macro-F1 (fastText + compact.vec) =", f1_score(y_true, y_pred, average="macro"))


macro-F1 (fastText + compact.vec) = 0.6055693686312081


In [ ]:
import fasttext, os
# если предыдущая модель «повисла», перезапустите ядро и импортируйте fasttext заново

model_pre = fasttext.train_supervised(
    input              = "train.txt",
    pretrainedVectors  = "compact.vec",
    epoch              = 80,      # побольше эпох
    lr                 = 0.3,     # пониже скорость
    wordNgrams         = 3,       # захватываем 1–3-граммы
    dim                = 300,     # ↓ RAM ×3, кач. почти не падает
    minn               = 2,       # субсловные n-граммы
    maxn               = 5,
    bucket             = 2_000_000,  # стандарт
    loss               = "ova",
    thread             = 1
)
model_pre.save_model("ft_pre.bin")


In [ ]:
from sklearn.metrics import f1_score

model_pre = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
with open("test.txt", encoding="utf-8") as f:
    for ln in f:
        lab,*txt = ln.strip().split()
        y_true.append(lab.replace("__label__",""))
        y_pred.append(
            model_pre.predict(" ".join(txt))[0][0].replace("__label__","")
        )

print("✅ macro-F1 (fastText pretrained, dim 300) =",
      f1_score(y_true, y_pred, average="macro"))


✅ macro-F1 (fastText pretrained, dim 300) = 0.6117849163593772


In [ ]:
import fasttext

model_pre = fasttext.train_supervised(
    input              = "train.txt",
    pretrainedVectors  = "compact.vec",
    epoch              = 80,     # больше эпох
    lr                 = 0.3,    # мягче шаг
    wordNgrams         = 3,      # 1-3-граммы
    dim                = 300,    # можно 100, но 300 OK
    minn               = 2,
    maxn               = 5,
    loss               = "ova",
    thread             = 1
)
model_pre.save_model("ft_pre.bin")

In [ ]:
from sklearn.metrics import f1_score, classification_report
import fasttext, re, pandas as pd

model_pre = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace("__label__",""))
    y_pred.append(model_pre.predict(" ".join(txt))[0][0].replace("__label__",""))

print("macro-F1 =", f1_score(y_true, y_pred, average="macro"))


macro-F1 = 0.6071352559208687


In [ ]:
import fasttext

model_pre = fasttext.train_supervised(
    input             = "train.txt",
    pretrainedVectors = "compact.vec",
    epoch             = 100,      # ← дольше учим
    lr                = 0.25,     # ← шаг поменьше
    wordNgrams        = 4,        # ← учитываем 1–4-граммы
    dim               = 300,      # (можно 150, экономит ещё ~200 МБ)
    minn              = 2,        # субсловные фичи
    maxn              = 6,
    loss              = "ova",
    thread            = 1         # чтобы не раздувать RAM-пик
)
model_pre.save_model("ft_pre.bin")


In [ ]:
from sklearn.metrics import f1_score
import fasttext, re

model_pre = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace("__label__",""))
    y_pred.append(model_pre.predict(" ".join(txt))[0][0].replace("__label__",""))

print("macro-F1 =", f1_score(y_true, y_pred, average="macro"))


macro-F1 = 0.6113036679754708


In [ ]:
import fasttext

model_pre = fasttext.train_supervised(
    input             ="train.txt",
    pretrainedVectors ="compact.vec",
    epoch             =120,    # 1
    lr                =0.2,    # 1
    wordNgrams        =5,      # 2
    dim               =300,
    minn              =3,      # 3
    maxn              =6,      # 3
    bucket            =2000000,
    loss              ="ova",
    thread            =1
)
model_pre.save_model("ft_pre.bin")


In [ ]:
from sklearn.metrics import f1_score, classification_report
import fasttext, re

model_pre = fasttext.load_model("ft_pre.bin")

y_true, y_pred = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_true.append(lab.replace("__label__",""))
    y_pred.append(
        model_pre.predict(" ".join(txt))[0][0].replace("__label__","")
    )

print("macro-F1 (pretrained, tuned) =",
      f1_score(y_true, y_pred, average="macro"))


macro-F1 (pretrained, tuned) = 0.6148823170148076


In [ ]:
import fasttext, gc
del model_pre ; gc.collect()     # освободим память

model_pre = fasttext.train_supervised(
    input            = "train.txt",
    pretrainedVectors= "compact.vec",
    epoch            = 120,
    lr               = 0.25,
    wordNgrams       = 5,
    dim              = 300,
    minn             = 3,
    maxn             = 6,
    loss             = "ova",
    thread           = 1
)
model_pre.save_model("ft_pre.bin")


In [ ]:
from sklearn.metrics import f1_score, classification_report
import fasttext, re

def macro_f1(model_path):
    mdl = fasttext.load_model(model_path)
    y_t, y_p = [], []
    for ln in open("test.txt", encoding="utf-8"):
        lab,*txt = ln.strip().split()
        y_t.append(lab.replace("__label__",""))
        y_p.append(mdl.predict(" ".join(txt))[0][0].replace("__label__",""))
    return f1_score(y_t, y_p, average="macro")

print("macro-F1 (pretrained tuned) =", macro_f1("ft_pre.bin"))
# или ft_pre_ft.bin / ft_pre_auto.bin


macro-F1 (pretrained tuned) = 0.6132125266275565


In [ ]:
import re, pandas as pd, collections, gzip, json

def clean(t): return re.sub(r'\W+', ' ', str(t).lower()).strip()

df = pd.read_csv('rusentiment_random_posts.csv')
freq = collections.Counter()

for txt in df['text'].map(clean):
    freq.update(txt.split())

# сохраняем частоты (на будущее)
with open('word_freq.json', 'w', encoding='utf-8') as f:
    json.dump(freq.most_common(), f, ensure_ascii=False)


In [ ]:
TOP = 300_000                               # сколько слов оставить
need = {w for w, _ in freq.most_common(TOP)}

with open("cc.ru.300.vec\cc.ru.300.vec", encoding="utf-8") as src, \
     open("compact300k.vec", "w", encoding="utf-8") as dst:
    total, dim = map(int, src.readline().split())
    kept = [ln for ln in src if ln.split(' ',1)[0] in need]
    dst.write(f"{len(kept)} {dim}\n");  dst.writelines(kept)

print("compact300k.vec сохранён, слов:", len(kept))



compact300k.vec сохранён, слов: 42878


In [ ]:
import fasttext

model_pre = fasttext.train_supervised(
    input            ="train.txt",
    pretrainedVectors="compact300k.vec",
    epoch            =80,
    lr               =0.3,
    wordNgrams       =4,
    dim              =300,      # если хватает RAM, можно 300
    minn             =2,
    maxn             =5,
    loss             ="ova",
    thread           =1
)
model_pre.save_model("ft_pre.bin")


In [ ]:
from sklearn.metrics import f1_score
import fasttext, re

m = fasttext.load_model("ft_pre.bin")     # или .bin из CLI
y_t, y_p = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_t.append(lab.replace("__label__",""))
    y_p.append(m.predict(" ".join(txt))[0][0].replace("__label__",""))
print("macro-F1 =", f1_score(y_t, y_p, average="macro"))

✅ macro-F1 (fastText): 0.625394757667914414


In [ ]:
from sklearn.metrics import f1_score
import fasttext, re

m = fasttext.load_model("ft_pre.bin")     # или .bin из CLI
y_t, y_p = [], []
for ln in open("test.txt", encoding="utf-8"):
    lab,*txt = ln.strip().split()
    y_t.append(lab.replace("__label__",""))
    y_p.append(m.predict(" ".join(txt))[0][0].replace("__label__",""))
print("macro-F1 =", f1_score(y_t, y_p, average="macro"))

macro-F1 = 0.6153947576679144


# 8 Автоматическая классификация депрессивных текстов

In [ ]:
депрессивность отличать депрессиии

In [ ]:
https://github.com/Ru-Psychology/russian-depressive-posts

In [ ]:
import urllib.request

url = "https://github.com/Ru-Psychology/russian-depressive-posts"
urllib.request.urlretrieve(url, "russian-depressive-posts.csv")

In [ ]:
!pip install pyspellchecker



   ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.2 MB 2.8 MB/s eta 0:00:03
   ---------- ----------------------------- 1.8/7.2 MB 5.9 MB/s eta 0:00:01
   ----------- ---------------------------- 2.1/7.2 MB 4.5 MB/s eta 0:00:02
   ------------- -------------------------- 2.4/7.2 MB 3.4 MB/s eta 0:00:02
   -------------- ------------------------- 2.6/7.2 MB 2.9 MB/s eta 0:00:02
   ----------------- ---------------------- 3.1/7.2 MB 2.4 MB/s eta 0:00:02
   ------------------ --------------------- 3.4/7.2 MB 2.2 MB/s eta 0:00:02
   -------------------- ------------------- 3.7/7.2 MB 2.1 MB/s eta 0:00:02
   --------------------- ------------------ 3.9/7.2 MB 2.0 MB/s eta 0:00:02
   ----------------------- ---------------- 4.2/7.2 MB 2.0 MB/s eta 0:00:02
   ------------------------ --------------- 4.5/7.2 MB 1.9 MB/s eta 0:00:02
   -----------------------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install stopwordsiso


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!pip install openpyxl



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
!swig -version


ERROR: Could not find a version that satisfies the requirement jamspell-windows (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for jamspell-windows


In [ ]:
"""
Лабораторная № 8  —  Автоматическая классификация депрессивных текстов
(«норма / подавленность / клиническая депрессия»).

● Корпус: Russian Depressive Posts (32 018 + 32 021)
  └─ https://github.com/Ru-Psychology/russian-depressive-posts  (CSV ≈ 45 МБ)

Pipeline = 7 предыдущих работ:
  1) RegExp-чистка + «ё→е»
  2) Токенизация   (razdel)
  3) Коррекция     (jamspell)
  4) Морфология    (pymorphy2 → леммы)
  5) Синтаксис-признак «НЕ-модальный глагол»
  6) NER-признак   (упоминание психотропного лекарства)
  7) RuSentiLex-признаки + TF-IDF 1–2-грамм
"""

# ───────────────────────  INSTALL / IMPORTS  ────────────────────────────
# pip install razdel jamspell pymorphy2 stopwordsiso stop-words
# pip install scikit-learn pandas numpy tqdm

import re, os, json, gzip, itertools, numpy as np, pandas as pd, joblib, string
from pathlib import Path
from tqdm.auto import tqdm
from razdel import tokenize as raz_tokenize
from spellchecker import SpellChecker
# from jamspell import TSpellCorrector
from stopwordsiso import stopwords as sw_iso
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack, csr_matrix
import pymorphy2
from spellchecker import SpellChecker
import re

# ───────────────────────  0. DATA  ──────────────────────────────────────
# DATA_URL = "https://huggingface.co/datasets/RuPsych/russian_depressive_posts/resolve/main/rus_depr_train.csv"
DATA_CSV = "Depressive data.xlsx"
if not Path(DATA_CSV).exists():
    import urllib.request, ssl
    ssl._create_default_https_context = ssl._create_unverified_context
    urllib.request.urlretrieve(DATA_URL, DATA_CSV)

#df = pd.read_csv(DATA_CSV)     # колонки: text, label  (0-norm / 1-distress / 2-depr)
#df = pd.read_csv(DATA_CSV, encoding='cp1251')
#df = pd.read_csv(DATA_CSV, encoding='ISO-8859-1')
df = pd.read_excel("Depressive data.xlsx")  # пример

# ───────────────────────  1. RegExp clean  ─────────────────────────────
url_pat = re.compile(r"http\S+")
multi_pat = re.compile(r"(.)\1{3,}")        # аааа → аа
punct_pat = re.compile(rf"[^\w\s{string.punctuation}]+")

def clean(txt: str) -> str:
    txt = url_pat.sub(" ", str(txt).lower())
    txt = txt.replace("ё", "е")
    txt = multi_pat.sub(r"\1\1", txt)       # до двух подряд
    txt = punct_pat.sub(" ", txt)
    return re.sub(r"\s+", " ", txt).strip()

df["clean"] = df["text"].map(clean)

# ───────────────────────  2. Tok + 3. Spell  ───────────────────────────
spell = SpellChecker(language="ru")
def raz_tokenize(text):
    return re.findall(r'\w+', text.lower())

# Функция исправления опечаток и токенизации
def tok_spell(text: str):
    tokens = raz_tokenize(text)
    fixed = [spell.correction(t) for t in tokens]
    return fixed

# ───────────────────────  4. Lemmatize  ────────────────────────────────
morph = pymorphy2.MorphAnalyzer()
def lemmas(tokens):
    return [morph.parse(t)[0].normal_form for t in tokens]

# ───────────────────────  5. Syntax-feature  (не хочу, не могу …)  ────
neg_aux_re = re.compile(r"\bне\s+(хочу|могу|буду|нравится|способен)\b")
def neg_aux_feat(txt):
    return int(bool(neg_aux_re.search(txt)))

# ───────────────────────  6. NER-drug (simple list)  ───────────────────
DRUGS = {"флуоксетин","сертралин","пароксетин","анафранил",
         "амитриптилин","венлафаксин","селектра","золофт",
         "прозак","эглонил","асентра"}
def drug_feat(txt):
    return int(any(w in txt for w in DRUGS))

# ───────────────────────  7. RuSentiLex  ───────────────────────────────
if not Path("rusentilex_2017.txt").exists():
    import urllib.request, ssl, textwrap, io, zipfile, tempfile, pathlib, requests
    url = "https://www.labinform.ru/pub/rusentilex/rusentilex_2017.txt"
    urllib.request.urlretrieve(url, "rusentilex_2017.txt")

lex = {}
with open("rusentilex_2017.txt", encoding="utf-8") as f:
    for ln in f:
        if ln.startswith("!"): continue
        parts = [p.strip() for p in ln.split(",")]
        if len(parts)>=4:
            lemma, pol = parts[2].lower(), parts[3].lower()
            if pol in {"positive","negative"}: lex[lemma]=pol

def lex_feats(lems):
    pos = sum(1 for w in lems if lex.get(w)=="positive")
    neg = sum(1 for w in lems if lex.get(w)=="negative")
    total = len(lems) or 1
    return pos/total, neg/total

# ───────────────────────  STOP-LIST  ────────────────────────────────────
ru_stop = set(sw_iso("ru")) | {"это","как","когда","кто","чего","этот","ваш",
                               "наш","твой","день","свой","еще"}

# ───────────────────────  Фиче-функция  ────────────────────────────────
def build_features(texts):
    tokens_lst   = [tok_spell(t) for t in tqdm(texts, desc="tokenize+spell")]
    lemmas_lst   = [lemmas(toks) for toks in tqdm(tokens_lst, desc="lemmatize")]

    # TF-IDF на леммах (1–2-граммы)
    docs = [" ".join(l) for l in lemmas_lst]
    tfidf = TfidfVectorizer(ngram_range=(1,2),
                            max_features=60_000,
                            min_df=3,
                            stop_words=ru_stop,
                            sublinear_tf=True)
    X_tfidf = tfidf.fit_transform(docs)

    # дополнительные признаки
    extras = []
    for txt,lms in zip(texts,lemmas_lst):
        pos_r,neg_r = lex_feats(lms)
        extras.append([neg_aux_feat(txt), drug_feat(txt), pos_r, neg_r])
    X_ex = csr_matrix(extras, dtype=np.float32)

    return hstack([X_tfidf, X_ex]), tfidf

# ───────────────────────  Train / Test split  ──────────────────────────
from sklearn.model_selection import train_test_split
Xtrain, Xtest, ytrain, ytest = train_test_split(df["clean"], df["label"],
                                               test_size=0.15,
                                               random_state=42,
                                               stratify=df["label"])

# ─── строим фичи
X_train, tfidf = build_features(Xtrain.tolist())
X_test  = hstack([tfidf.transform(Xtest.tolist()),
                  csr_matrix([[neg_aux_feat(t),drug_feat(t),*lex_feats(lemmas(tok_spell(t)))]
                              for t in Xtest], dtype=np.float32)])

# ───────────────────────  Модель  ──────────────────────────────────────
clf = LogisticRegression(max_iter=1500,
                         class_weight='balanced',
                         solver='saga', C=4.0, n_jobs=1)
clf.fit(X_train, ytrain)

ypred = clf.predict(X_test)
print(classification_report(ytest, ypred, digits=3))
print("macro-F1 =", f1_score(ytest, ypred, average="macro"))


tokenize+spell:   0%|          | 0/54433 [00:00<?, ?it/s]

KeyboardInterrupt: 